In [ ]:
# Working Forensic TrOCR GUI - Fixed Version
# Run this AFTER your forensic system is set up

import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import threading
import queue
import os
from datetime import datetime

class SimplifiedForensicGUI:
    def __init__(self, root, forensic_system=None, forensic_interface=None):
        self.root = root
        self.root.title("🔍 Forensic Image Search Interface (TrOCR)")
        self.root.geometry("1000x700")

        # System references
        self.forensic_system = forensic_system
        self.forensic_interface = forensic_interface
        self.current_results = None

        # Thread communication
        self.queue = queue.Queue()

        # Setup GUI
        self.setup_gui()

        # Check system status
        self.check_system_status()

        # Start queue processing
        self.process_queue()

    def setup_gui(self):
        """Create the GUI interface"""

        # Main container
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

        # Configure grid
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(0, weight=1)
        main_frame.columnconfigure(1, weight=1)
        main_frame.rowconfigure(1, weight=1)

        # Title
        title_label = ttk.Label(main_frame, text="🔍 Forensic Image Search Interface (TrOCR)",
                               font=('Arial', 16, 'bold'))
        title_label.grid(row=0, column=0, columnspan=2, pady=(0, 20))

        # Left Panel - Controls
        self.setup_controls(main_frame)

        # Right Panel - Results
        self.setup_results(main_frame)

    def setup_controls(self, parent):
        """Setup control panel"""
        controls_frame = ttk.LabelFrame(parent, text="🛠️ Search Controls", padding="15")
        controls_frame.grid(row=1, column=0, sticky=(tk.W, tk.E, tk.N, tk.S), padx=(0, 10))

        row = 0

        # Model display
        ttk.Label(controls_frame, text="Model:", font=('Arial', 10, 'bold')).grid(row=row, column=0, sticky=tk.W, pady=(0, 5))
        row += 1
        model_label = ttk.Label(controls_frame, text="TrOCR-Base-Printed - Best balance of speed and accuracy",
                               foreground='blue')
        model_label.grid(row=row, column=0, sticky=tk.W, pady=(0, 15))
        row += 1

        # Query input
        ttk.Label(controls_frame, text="Query:", font=('Arial', 10, 'bold')).grid(row=row, column=0, sticky=tk.W, pady=(0, 5))
        row += 1
        self.query_var = tk.StringVar(value="bank account")
        query_entry = ttk.Entry(controls_frame, textvariable=self.query_var, width=40, font=('Arial', 11))
        query_entry.grid(row=row, column=0, sticky=(tk.W, tk.E), pady=(0, 15))
        row += 1

        # Min Confidence
        ttk.Label(controls_frame, text="Min Confidence:", font=('Arial', 10, 'bold')).grid(row=row, column=0, sticky=tk.W, pady=(0, 5))
        row += 1
        conf_frame = ttk.Frame(controls_frame)
        conf_frame.grid(row=row, column=0, sticky=(tk.W, tk.E), pady=(0, 10))

        self.confidence_var = tk.DoubleVar(value=0.35)
        confidence_scale = ttk.Scale(conf_frame, from_=0.1, to=0.9,
                                   variable=self.confidence_var, orient=tk.HORIZONTAL, length=200)
        confidence_scale.grid(row=0, column=0, sticky=(tk.W, tk.E))

        self.confidence_label = ttk.Label(conf_frame, text="0.35", font=('Arial', 9, 'bold'))
        self.confidence_label.grid(row=0, column=1, padx=(10, 0))

        confidence_scale.configure(command=self.update_confidence_label)
        conf_frame.columnconfigure(0, weight=1)
        row += 1

        # Batch Size
        ttk.Label(controls_frame, text="Batch Size:", font=('Arial', 10, 'bold')).grid(row=row, column=0, sticky=tk.W, pady=(0, 5))
        row += 1
        batch_frame = ttk.Frame(controls_frame)
        batch_frame.grid(row=row, column=0, sticky=(tk.W, tk.E), pady=(0, 15))

        self.batch_var = tk.IntVar(value=4)
        batch_scale = ttk.Scale(batch_frame, from_=1, to=16,
                              variable=self.batch_var, orient=tk.HORIZONTAL, length=200)
        batch_scale.grid(row=0, column=0, sticky=(tk.W, tk.E))

        self.batch_label = ttk.Label(batch_frame, text="4", font=('Arial', 9, 'bold'))
        self.batch_label.grid(row=0, column=1, padx=(10, 0))

        batch_scale.configure(command=self.update_batch_label)
        batch_frame.columnconfigure(0, weight=1)
        row += 1

        # Buttons
        button_frame = ttk.Frame(controls_frame)
        button_frame.grid(row=row, column=0, sticky=(tk.W, tk.E), pady=(10, 0))
        row += 1

        self.search_btn = ttk.Button(button_frame, text="🔍 Search Gallery",
                                    command=self.search_gallery, state="disabled")
        self.search_btn.grid(row=0, column=0, columnspan=2, sticky=(tk.W, tk.E), pady=(0, 8))

        self.copy_btn = ttk.Button(button_frame, text="📋 Copy Results",
                                  command=self.copy_results, state="disabled")
        self.copy_btn.grid(row=1, column=0, sticky=(tk.W, tk.E), padx=(0, 5))

        self.clear_btn = ttk.Button(button_frame, text="🗑️ Clear Results",
                                   command=self.clear_results)
        self.clear_btn.grid(row=1, column=1, sticky=(tk.W, tk.E), padx=(5, 0))

        button_frame.columnconfigure((0, 1), weight=1)

        # Status
        status_frame = ttk.LabelFrame(controls_frame, text="📊 System Status", padding="10")
        status_frame.grid(row=row, column=0, sticky=(tk.W, tk.E, tk.N, tk.S), pady=(15, 0))
        controls_frame.rowconfigure(row, weight=1)

        self.status_text = scrolledtext.ScrolledText(status_frame, height=12, width=40,
                                                    font=('Consolas', 9))
        self.status_text.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        status_frame.columnconfigure(0, weight=1)
        status_frame.rowconfigure(0, weight=1)

        # Configure control frame column
        controls_frame.columnconfigure(0, weight=1)

    def setup_results(self, parent):
        """Setup results panel"""
        results_frame = ttk.LabelFrame(parent, text="🔍 Search Results", padding="15")
        results_frame.grid(row=1, column=1, sticky=(tk.W, tk.E, tk.N, tk.S))
        results_frame.columnconfigure(0, weight=1)
        results_frame.rowconfigure(1, weight=1)

        # Results info
        self.results_info_var = tk.StringVar(value="No search performed yet")
        info_label = ttk.Label(results_frame, textvariable=self.results_info_var,
                              font=('Arial', 11, 'bold'), foreground='blue')
        info_label.grid(row=0, column=0, sticky=tk.W, pady=(0, 10))

        # Results table
        self.setup_results_table(results_frame)

    def setup_results_table(self, parent):
        """Setup results table"""
        table_frame = ttk.Frame(parent)
        table_frame.grid(row=1, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        table_frame.columnconfigure(0, weight=1)
        table_frame.rowconfigure(0, weight=1)

        # Treeview
        columns = ('Score', 'Type', 'Confidence', 'Preview')
        self.results_tree = ttk.Treeview(table_frame, columns=columns, show='tree headings', height=20)
        self.results_tree.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

        # Configure columns
        self.results_tree.heading('#0', text='Filename')
        self.results_tree.heading('Score', text='Similarity')
        self.results_tree.heading('Type', text='Search Type')
        self.results_tree.heading('Confidence', text='OCR Conf.')
        self.results_tree.heading('Preview', text='Text Preview')

        self.results_tree.column('#0', width=150)
        self.results_tree.column('Score', width=80)
        self.results_tree.column('Type', width=100)
        self.results_tree.column('Confidence', width=80)
        self.results_tree.column('Preview', width=300)

        # Scrollbar
        scrollbar = ttk.Scrollbar(table_frame, orient=tk.VERTICAL, command=self.results_tree.yview)
        scrollbar.grid(row=0, column=1, sticky=(tk.N, tk.S))
        self.results_tree.configure(yscrollcommand=scrollbar.set)

        # Bind double-click
        self.results_tree.bind('<Double-1>', self.on_result_double_click)

    def update_confidence_label(self, value):
        """Update confidence label"""
        self.confidence_label.config(text=f"{float(value):.2f}")

    def update_batch_label(self, value):
        """Update batch label"""
        self.batch_label.config(text=f"{int(float(value))}")

    def update_status(self, message):
        """Update status display"""
        timestamp = datetime.now().strftime("%H:%M:%S")
        formatted_message = f"[{timestamp}] {message}\n"

        self.status_text.insert(tk.END, formatted_message)
        self.status_text.see(tk.END)
        self.root.update_idletasks()

    def check_system_status(self):
        """Check if forensic system is available"""
        if self.forensic_system and self.forensic_interface:
            self.update_status("✅ Forensic system ready")
            self.search_btn.config(state="normal")

            # Display system info
            try:
                stats = self.forensic_system.get_system_statistics()
                self.update_status(f"📊 Images processed: {stats.get('total_images', 0)}")
                self.update_status(f"📝 Images with text: {stats.get('images_with_text', 0)}")
            except Exception as e:
                self.update_status(f"⚠️ Could not get stats: {str(e)[:50]}")
        else:
            self.update_status("❌ Forensic system not initialized")
            self.update_status("💡 Please run forensic system setup first")

    def search_gallery(self):
        """Search gallery"""
        query = self.query_var.get().strip()
        if not query:
            messagebox.showwarning("Warning", "Please enter a search query")
            return

        if not self.forensic_system:
            messagebox.showerror("Error", "Forensic system not available")
            return

        confidence = self.confidence_var.get()
        batch_size = self.batch_var.get()

        def search_worker():
            try:
                # Update status
                self.queue.put(("status", f"🔍 Searching for: '{query}'"))
                self.queue.put(("status", f"⚙️ Confidence: {confidence}, Batch: {batch_size}"))

                # Update system settings
                self.forensic_system.update_processing_settings(
                    similarity_threshold=confidence,
                    batch_size=batch_size
                )

                # Perform search
                results = self.forensic_system.search_gallery(query)

                # Send results back to main thread
                self.queue.put(("search_complete", results))

            except Exception as e:
                self.queue.put(("error", f"Search failed: {str(e)}"))

        # Disable search button
        self.search_btn.config(state="disabled")
        self.update_status("🔄 Search in progress...")

        # Start search in background
        thread = threading.Thread(target=search_worker, daemon=True)
        thread.start()

    def display_results(self, results):
        """Display search results"""
        if not results or not results.get('combined_results'):
            self.results_info_var.set("No matches found")
            self.clear_results_table()
            return

        combined_results = results['combined_results']
        total_matches = results.get('total_matches', len(combined_results))
        processing_time = results.get('processing_time', 0)

        # Update info
        self.results_info_var.set(f"Found {total_matches} matches in {processing_time:.2f}s")

        # Store current results
        self.current_results = results
        self.copy_btn.config(state="normal")

        # Clear and populate table
        self.clear_results_table()

        for result in combined_results:
            score = result.get('similarity_score', result.get('ocr_confidence', 0))
            confidence = result.get('ocr_confidence', 0)
            search_type = result.get('search_type', 'unknown')
            preview = result.get('text_preview', '')
            filename = result.get('filename', 'Unknown')

            # Limit preview length
            if len(preview) > 80:
                preview = preview[:80] + "..."

            self.results_tree.insert('', 'end',
                                   text=filename,
                                   values=(f"{score:.3f}",
                                          search_type.title(),
                                          f"{confidence:.3f}",
                                          preview))

        # Update status
        self.update_status(f"✅ Found {len(combined_results)} matches")

    def clear_results_table(self):
        """Clear results table"""
        for item in self.results_tree.get_children():
            self.results_tree.delete(item)

    def copy_results(self):
        """Copy results to folder"""
        if not self.current_results:
            messagebox.showwarning("Warning", "No results to copy")
            return

        try:
            output_path = self.forensic_system.copy_results(self.current_results)
            if output_path:
                self.update_status(f"📋 Results copied to: {output_path}")
                messagebox.showinfo("Success", f"Results copied to:\n{output_path}")
            else:
                messagebox.showerror("Error", "Failed to copy results")
        except Exception as e:
            messagebox.showerror("Error", f"Copy failed: {str(e)}")

    def clear_results(self):
        """Clear results"""
        self.current_results = None
        self.copy_btn.config(state="disabled")
        self.clear_results_table()
        self.results_info_var.set("No search performed yet")
        self.update_status("🗑️ Results cleared")

    def on_result_double_click(self, event):
        """Handle double-click on result"""
        selection = self.results_tree.selection()
        if selection:
            item = self.results_tree.item(selection[0])
            filename = item['text']

            # Show detailed info
            if self.current_results:
                for result in self.current_results.get('combined_results', []):
                    if result.get('filename') == filename:
                        info = f"""File: {filename}
Search Type: {result.get('search_type', 'unknown')}
Similarity Score: {result.get('similarity_score', 0):.3f}
OCR Confidence: {result.get('ocr_confidence', 0):.3f}
Path: {result.get('full_path', 'N/A')}

Extracted Text:
{result.get('extracted_text', 'No text available')}"""

                        # Create info window
                        info_window = tk.Toplevel(self.root)
                        info_window.title(f"Details: {filename}")
                        info_window.geometry("600x400")

                        text_widget = scrolledtext.ScrolledText(info_window, wrap=tk.WORD, padding=10)
                        text_widget.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
                        text_widget.insert(1.0, info)
                        text_widget.config(state=tk.DISABLED)
                        break

    def process_queue(self):
        """Process messages from background threads"""
        try:
            while True:
                message_type, data = self.queue.get_nowait()

                if message_type == "status":
                    self.update_status(data)
                elif message_type == "error":
                    self.update_status(f"❌ {data}")
                    messagebox.showerror("Error", data)
                    self.search_btn.config(state="normal")
                elif message_type == "search_complete":
                    self.display_results(data)
                    self.search_btn.config(state="normal")

        except queue.Empty:
            pass

        # Schedule next check
        self.root.after(100, self.process_queue)

def launch_gui():
    """Launch the GUI"""
    # Try to get forensic system from globals
    forensic_sys = globals().get('forensic_system')
    forensic_int = globals().get('forensic_interface')

    if not forensic_sys or not forensic_int:
        print("⚠️ Forensic system not found in globals")
        print("💡 Make sure to run the forensic system setup first:")
        print("   forensic_system, forensic_interface = initialize_forensic_system()")
        print("")
        print("🚀 Launching GUI anyway for demonstration...")

    root = tk.Tk()
    app = SimplifiedForensicGUI(root, forensic_sys, forensic_int)

    print("🖥️ GUI launched successfully!")
    print("📋 Features:")
    print("  • Parameter controls (confidence, batch size)")
    print("  • Real-time search with progress updates")
    print("  • Results table with detailed information")
    print("  • Double-click results for full details")
    print("  • One-click result export")

    root.mainloop()

# Run this to launch the GUI
if __name__ == "__main__":
    launch_gui()